In [131]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Математическая модель

$productivity_i$ - мощности предприятий (тыс. т) для i = {0, 1, 2, 3, 4}

$Cost_i$ - удельная стоимость (тыс. р/тыс. т) для i = {0, 1, 2, 3, 4}

$order_i$ - заказы (тыс. т)  i = {0, 1, 2, 3}

$storageCapacity_i$ - вместимость склада (тыс. т) для для i = {0, 1, 2}

$D1$ - пропускная способность коммуникаций предприятие-склад

$D3$ - пропускная способность сортировка-потребитель

$C1$ - удельная стоимость перевозки предприятие-склад

$C2$ - удельная стоимость перевозки склад-сортировка

$C3$ - удельная стоимость перевозки сортировка-потребитель

Вводим переменные:

$x_i$ - план производства i-ого завода для i = {0, 1, 2, 3, 4}

$factory-working_i$ - булева переменная, которая равна единице, если предприятие работает и нулю иначе

$Transfer1_{ij} $- объем транспортировки с i-ого завода на j-ый склад

$Transfer2_{ij}$ -объем транспортировки с i-ого склада на j-ую сортировку

$Transfer3_{ij}$ - объем транспортировки с i-ой сортировки на j-ого потребителя

# Ограничения

$x_i <= productivity_i$ - произвли не больше мощности предприятия

$\sum_{i=0}^{4} factory-working_{i} \leqslant 2$ - работает не более двух предприятий

$x_i <= 1000000*factory-working_{i}$ - если предприятие не работает, то выпускает ноль

$Transfer1_{ij} <= D1_{ij}$ - ограничение на транспортировку с завода на склад

$Transfer3_{ij} <= D3_{ij}$ - ограничение на транспортировку со станций до потребителей

$\sum_{i=0}^{4} Transfer1_{ij} <= storageCapacity_i$ - ограничение на вместимость

$x_i = \sum_{j=0}^{2}Transfer1_{ij}$ - произведенная продукция вся отправляется на склады

$\sum_{i=0}^{4} Transfer1_{ij} = \sum_{k=0}^{1} Transfer2_{jk}$ - все со складов отправляется на сортировку

$\sum_{i=0}^{1} Transfer2_{ij} = \sum_{k=0}^{3} Transfer3_{jk}$ - все с сортировки отправляется до потребителя

$\sum_{i=0}^{1} Transfer3_{ij} = order_j$ - с обеих сортировок пришла продукция и суммарно равна заказу

# Целевая функция:

$\sum_{i=0}^{4} x_i*Cost_i + \sum_{over\;all\;ij} Transfer1_{ij}C1_{ij} + \sum_{over\;all\;ij} Transfer2_{ij}C2_{ij} + \sum_{over\;all\;ij} Transfer3_{ij}C3_{ij} \rightarrow min$

In [132]:
productivity = np.array([600, 500, 700, 400, 600])
cost = np.array([1.3, 1.5, 1.2, 1.7, 1.1])

order = np.array([100, 200, 200, 500])

storage_capacity = np.array([200, 500, 400])

D1 =  np.array([[150, 500, 0],
      [100, 0, 400],
      [50, 50, 0],
      [0, 100, 100],
      [100, 0, 50]])

D3 = np.array([[50, 50, 10, 300],
      [70, 150, 200, 250]])

C1 = np.array([[1, 2, 0],
      [2, 0, 1],
      [3, 3, 0],
      [0, 3, 2],
      [2, 0, 3]])

C2 = np.array([[2, 3],
      [1, 4],
      [3, 1]])

C3 = np.array([[2, 1, 3, 4],
      [3, 2, 3, 1]])

In [133]:
model = gp.Model('factoryProblem')

x = model.addVars(5, ub=productivity, name='x')

Transfer1 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), 
                                        (1,0), (1,1), (1,2),
                                        (2,0), (2,1), (2,2),
                                        (3,0), (3,1), (3,2),
                                        (4,0), (4,1), (4,2)]), ub = D1, name = 'Transfer1')

Transfer2 = model.addVars(gp.tuplelist([(0,0), (0,1), 
                                        (1,0), (1,1), 
                                        (2,0), (2,1)]), name = 'Transfer2')

Transfer3 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                        (1,0), (1,1), (1,2), (1,3) ]), ub = D3, name = 'Transfer3')

factory_working = model.addVars(5, vtype=GRB.BINARY, name='factory-working')

for i in range(5):
    model.addConstr(x[i] <= 1000000*factory_working[i], name='factory-working')
    
model.addConstr(factory_working.sum() <= 2)


<gurobi.Constr *Awaiting Model Update*>

In [134]:
model.addConstrs((x[i] == Transfer1.sum(i,'*') for i in range(5)), name = 'from factory to storage')

model.addConstrs((Transfer1.sum('*', i) 
                 == Transfer2.sum(i, '*') for i in range(3)), name = 'from storage to sorting')

model.addConstrs((Transfer2.sum('*', i) 
                 == Transfer3.sum(i, '*') for i in range(2)), name = 'from sorting to buyers')

model.addConstrs((Transfer1.sum('*', i) <= storage_capacity[i] for i in range(3)), name = 'storage capacity')

model.addConstrs((Transfer3.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders full')

obj = gp.LinExpr()

obj += sum([x[i]*cost[i] for i in range(5)]) + sum([Transfer1[i, j]*C1[i, j] for i in range(5) for j in range(3)]) +sum([Transfer2[i, j]*C2[i, j] for i in range(3) for j in range(2)]) + sum([Transfer3[i, j]*C3[i, j] for i in range(2) for j in range(4)])

model.setObjective(obj, GRB.MINIMIZE)

model.update()
model.display()

Minimize
   <gurobi.LinExpr: 1.3 x[0] + 1.5 x[1] + 1.2 x[2] + 1.7 x[3] + 1.1 x[4] + Transfer1[0,0] + 2.0 Transfer1[0,1] + 2.0 Transfer1[1,0] + Transfer1[1,2] + 3.0 Transfer1[2,0] + 3.0 Transfer1[2,1] + 3.0 Transfer1[3,1] + 2.0 Transfer1[3,2] + 2.0 Transfer1[4,0] + 3.0 Transfer1[4,2] + 2.0 Transfer2[0,0] + 3.0 Transfer2[0,1] + Transfer2[1,0] + 4.0 Transfer2[1,1] + 3.0 Transfer2[2,0] + Transfer2[2,1] + 2.0 Transfer3[0,0] + Transfer3[0,1] + 3.0 Transfer3[0,2] + 4.0 Transfer3[0,3] + 3.0 Transfer3[1,0] + 2.0 Transfer3[1,1] + 3.0 Transfer3[1,2] + Transfer3[1,3]>
Subject To
   factory-working : <gurobi.LinExpr: x[0] + -1000000.0 factory-working[0]> <= 0.0
   factory-working : <gurobi.LinExpr: x[1] + -1000000.0 factory-working[1]> <= 0.0
   factory-working : <gurobi.LinExpr: x[2] + -1000000.0 factory-working[2]> <= 0.0
   factory-working : <gurobi.LinExpr: x[3] + -1000000.0 factory-working[3]> <= 0.0
   factory-working : <gurobi.LinExpr: x[4] + -1000000.0 factory-working[4]> <= 0.0
   R5 : <gu

In [135]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 23 rows, 39 columns and 93 nonzeros
Model fingerprint: 0x77f1f99c
Variable types: 34 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 7e+02]
  RHS range        [2e+00, 5e+02]
Presolve removed 9 rows and 15 columns
Presolve time: 0.00s
Presolved: 14 rows, 24 columns, 60 nonzeros
Variable types: 19 continuous, 5 integer (5 binary)

Root relaxation: objective 6.805000e+03, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6805.00000    0    2          - 6805.00000      -     -    0s
H    0     0                    6810.0000000 6805.00000  0.07%     -    0s

Explored 1 nodes (11 simplex iterations)

In [136]:
model.getVars()

[<gurobi.Var x[0] (value 550.0)>,
 <gurobi.Var x[1] (value 450.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 0.0)>,
 <gurobi.Var Transfer1[0,0] (value 150.0)>,
 <gurobi.Var Transfer1[0,1] (value 400.0)>,
 <gurobi.Var Transfer1[0,2] (value 0.0)>,
 <gurobi.Var Transfer1[1,0] (value 50.0)>,
 <gurobi.Var Transfer1[1,1] (value 0.0)>,
 <gurobi.Var Transfer1[1,2] (value 400.0)>,
 <gurobi.Var Transfer1[2,0] (value 0.0)>,
 <gurobi.Var Transfer1[2,1] (value 0.0)>,
 <gurobi.Var Transfer1[2,2] (value 0.0)>,
 <gurobi.Var Transfer1[3,0] (value 0.0)>,
 <gurobi.Var Transfer1[3,1] (value 0.0)>,
 <gurobi.Var Transfer1[3,2] (value 0.0)>,
 <gurobi.Var Transfer1[4,0] (value 0.0)>,
 <gurobi.Var Transfer1[4,1] (value 0.0)>,
 <gurobi.Var Transfer1[4,2] (value 0.0)>,
 <gurobi.Var Transfer2[0,0] (value 0.0)>,
 <gurobi.Var Transfer2[0,1] (value 200.0)>,
 <gurobi.Var Transfer2[1,0] (value 400.0)>,
 <gurobi.Var Transfer2[1,1] (value 0.0)>,
 <gurobi.Var Transfer2[2,0]

# Задание 2. На первом складе уже есть 100 тонн
Что изменится: ограничение для доставки на первый склад теперь будет меньше на 100, но при этом эти же 100 можно доставлять покупателям

In [137]:
storage_already = np.array([100,0,0])

model = gp.Model('factoryProblem2')

x = model.addVars(5, ub=productivity, name='x')

Transfer1 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), 
                                        (1,0), (1,1), (1,2),
                                        (2,0), (2,1), (2,2),
                                        (3,0), (3,1), (3,2),
                                        (4,0), (4,1), (4,2)]), ub = D1, name = 'Transfer1')

Transfer2 = model.addVars(gp.tuplelist([(0,0), (0,1), 
                                        (1,0), (1,1), 
                                        (2,0), (2,1)]), name = 'Transfer2')

Transfer3 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                        (1,0), (1,1), (1,2), (1,3) ]), ub = D3, name = 'Transfer3')

factory_working = model.addVars(5, vtype=GRB.BINARY, name='factory-working')

for i in range(5):
    model.addConstr(x[i] <= 1000000*factory_working[i], name='factory-working')
    
model.addConstr(factory_working.sum() <= 2)



<gurobi.Constr *Awaiting Model Update*>

In [138]:
model.addConstrs((x[i] == Transfer1.sum(i,'*') for i in range(5)), name = 'from factory to storage')

model.addConstrs((Transfer1.sum('*', i) + storage_already[i]
                 == Transfer2.sum(i, '*') for i in range(3)), name = 'from storage to sorting')

model.addConstrs((Transfer2.sum('*', i) 
                 == Transfer3.sum(i, '*') for i in range(2)), name = 'from sorting to buyers')

model.addConstrs((Transfer1.sum('*', i) <= storage_capacity[i] - storage_already[i] for i in range(3)), name = 'storage capacity')

model.addConstrs((Transfer3.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders full')

obj = gp.LinExpr()

obj += sum([x[i]*cost[i] for i in range(5)]) + sum([Transfer1[i, j]*C1[i, j] for i in range(5) for j in range(3)]) +sum([Transfer2[i, j]*C2[i, j] for i in range(3) for j in range(2)]) + sum([Transfer3[i, j]*C3[i, j] for i in range(2) for j in range(4)])

model.setObjective(obj, GRB.MINIMIZE)

model.update()
model.display()

Minimize
   <gurobi.LinExpr: 1.3 x[0] + 1.5 x[1] + 1.2 x[2] + 1.7 x[3] + 1.1 x[4] + Transfer1[0,0] + 2.0 Transfer1[0,1] + 2.0 Transfer1[1,0] + Transfer1[1,2] + 3.0 Transfer1[2,0] + 3.0 Transfer1[2,1] + 3.0 Transfer1[3,1] + 2.0 Transfer1[3,2] + 2.0 Transfer1[4,0] + 3.0 Transfer1[4,2] + 2.0 Transfer2[0,0] + 3.0 Transfer2[0,1] + Transfer2[1,0] + 4.0 Transfer2[1,1] + 3.0 Transfer2[2,0] + Transfer2[2,1] + 2.0 Transfer3[0,0] + Transfer3[0,1] + 3.0 Transfer3[0,2] + 4.0 Transfer3[0,3] + 3.0 Transfer3[1,0] + 2.0 Transfer3[1,1] + 3.0 Transfer3[1,2] + Transfer3[1,3]>
Subject To
   factory-working : <gurobi.LinExpr: x[0] + -1000000.0 factory-working[0]> <= 0.0
   factory-working : <gurobi.LinExpr: x[1] + -1000000.0 factory-working[1]> <= 0.0
   factory-working : <gurobi.LinExpr: x[2] + -1000000.0 factory-working[2]> <= 0.0
   factory-working : <gurobi.LinExpr: x[3] + -1000000.0 factory-working[3]> <= 0.0
   factory-working : <gurobi.LinExpr: x[4] + -1000000.0 factory-working[4]> <= 0.0
   R5 : <gu

In [139]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 23 rows, 39 columns and 93 nonzeros
Model fingerprint: 0xbd6dfb88
Variable types: 34 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 7e+02]
  RHS range        [2e+00, 5e+02]
Presolve removed 9 rows and 15 columns
Presolve time: 0.00s
Presolved: 14 rows, 24 columns, 60 nonzeros
Variable types: 19 continuous, 5 integer (5 binary)

Root relaxation: objective 6.520000e+03, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6520.0000000 6520.00000  0.00%     -    0s

Explored 0 nodes (10 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution c

In [140]:
model.getVars()

[<gurobi.Var x[0] (value 500.0)>,
 <gurobi.Var x[1] (value 400.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 0.0)>,
 <gurobi.Var Transfer1[0,0] (value 100.0)>,
 <gurobi.Var Transfer1[0,1] (value 400.0)>,
 <gurobi.Var Transfer1[0,2] (value 0.0)>,
 <gurobi.Var Transfer1[1,0] (value 0.0)>,
 <gurobi.Var Transfer1[1,1] (value 0.0)>,
 <gurobi.Var Transfer1[1,2] (value 400.0)>,
 <gurobi.Var Transfer1[2,0] (value 0.0)>,
 <gurobi.Var Transfer1[2,1] (value 0.0)>,
 <gurobi.Var Transfer1[2,2] (value 0.0)>,
 <gurobi.Var Transfer1[3,0] (value 0.0)>,
 <gurobi.Var Transfer1[3,1] (value 0.0)>,
 <gurobi.Var Transfer1[3,2] (value 0.0)>,
 <gurobi.Var Transfer1[4,0] (value 0.0)>,
 <gurobi.Var Transfer1[4,1] (value 0.0)>,
 <gurobi.Var Transfer1[4,2] (value 0.0)>,
 <gurobi.Var Transfer2[0,0] (value 0.0)>,
 <gurobi.Var Transfer2[0,1] (value 200.0)>,
 <gurobi.Var Transfer2[1,0] (value 360.0)>,
 <gurobi.Var Transfer2[1,1] (value 40.0)>,
 <gurobi.Var Transfer2[2,0]

# Задание 3. Теперь есть затраты на организацию производства. 
У нас уже есть бинарные переменные для определения - работает фабрика или нет
Перемножим затраты и бинарные переменные и добавим в целевую функцию

In [156]:
factory_cost = np.array([1000, 2000, 3000, 2500, 1500])

model = gp.Model('factoryProblem3')

x = model.addVars(5, ub=productivity, name='x')

Transfer1 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), 
                                        (1,0), (1,1), (1,2),
                                        (2,0), (2,1), (2,2),
                                        (3,0), (3,1), (3,2),
                                        (4,0), (4,1), (4,2)]), ub = D1, name = 'Transfer1')

Transfer2 = model.addVars(gp.tuplelist([(0,0), (0,1), 
                                        (1,0), (1,1), 
                                        (2,0), (2,1)]), name = 'Transfer2')

Transfer3 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                        (1,0), (1,1), (1,2), (1,3) ]), ub = D3, name = 'Transfer3')

factory_working = model.addVars(5, vtype=GRB.BINARY, name='factory-working')

for i in range(5):
    model.addConstr(x[i] <= 1000000*factory_working[i], name='factory-working')

In [157]:
model.addConstrs((x[i] == Transfer1.sum(i,'*') for i in range(5)), name = 'from factory to storage')

model.addConstrs((Transfer1.sum('*', i)
                 == Transfer2.sum(i, '*') for i in range(3)), name = 'from storage to sorting')

model.addConstrs((Transfer2.sum('*', i) 
                 == Transfer3.sum(i, '*') for i in range(2)), name = 'from sorting to buyers')

model.addConstrs((Transfer1.sum('*', i) <= storage_capacity[i] for i in range(3)), name = 'storage capacity')

model.addConstrs((Transfer3.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders full')

obj = gp.LinExpr()

obj += sum([x[i]*cost[i] for i in range(5)]) + sum([Transfer1[i, j]*C1[i, j] for i in range(5) for j in range(3)]) +sum([Transfer2[i, j]*C2[i, j] for i in range(3) for j in range(2)]) + sum([Transfer3[i, j]*C3[i, j] for i in range(2) for j in range(4)]) + sum([factory_working[i]*factory_cost[i] for i in range(5)])

model.setObjective(obj, GRB.MINIMIZE)

model.update()
model.display()

Minimize
   <gurobi.LinExpr: 1.3 x[0] + 1.5 x[1] + 1.2 x[2] + 1.7 x[3] + 1.1 x[4] + Transfer1[0,0] + 2.0 Transfer1[0,1] + 2.0 Transfer1[1,0] + Transfer1[1,2] + 3.0 Transfer1[2,0] + 3.0 Transfer1[2,1] + 3.0 Transfer1[3,1] + 2.0 Transfer1[3,2] + 2.0 Transfer1[4,0] + 3.0 Transfer1[4,2] + 2.0 Transfer2[0,0] + 3.0 Transfer2[0,1] + Transfer2[1,0] + 4.0 Transfer2[1,1] + 3.0 Transfer2[2,0] + Transfer2[2,1] + 2.0 Transfer3[0,0] + Transfer3[0,1] + 3.0 Transfer3[0,2] + 4.0 Transfer3[0,3] + 3.0 Transfer3[1,0] + 2.0 Transfer3[1,1] + 3.0 Transfer3[1,2] + Transfer3[1,3] + 1000.0 factory-working[0] + 2000.0 factory-working[1] + 3000.0 factory-working[2] + 2500.0 factory-working[3] + 1500.0 factory-working[4]>
Subject To
   factory-working : <gurobi.LinExpr: x[0] + -1000000.0 factory-working[0]> <= 0.0
   factory-working : <gurobi.LinExpr: x[1] + -1000000.0 factory-working[1]> <= 0.0
   factory-working : <gurobi.LinExpr: x[2] + -1000000.0 factory-working[2]> <= 0.0
   factory-working : <gurobi.LinExpr:

In [158]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 39 columns and 88 nonzeros
Model fingerprint: 0x11b116e9
Variable types: 34 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 3e+03]
  Bounds range     [1e+00, 7e+02]
  RHS range        [1e+02, 5e+02]
Presolve removed 9 rows and 15 columns
Presolve time: 0.00s
Presolved: 13 rows, 24 columns, 55 nonzeros
Variable types: 19 continuous, 5 integer (5 binary)

Root relaxation: objective 9.600000e+03, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9600.00000    0    1          - 9600.00000      -     -    0s
H    0     0                    9810.0000000 9600.00000  2.14%     -    0s
     0     0     cutoff    0      9810.00

In [159]:
model.getVars()

[<gurobi.Var x[0] (value 550.0)>,
 <gurobi.Var x[1] (value 450.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 0.0)>,
 <gurobi.Var Transfer1[0,0] (value 150.0)>,
 <gurobi.Var Transfer1[0,1] (value 400.0)>,
 <gurobi.Var Transfer1[0,2] (value 0.0)>,
 <gurobi.Var Transfer1[1,0] (value 50.0)>,
 <gurobi.Var Transfer1[1,1] (value 0.0)>,
 <gurobi.Var Transfer1[1,2] (value 400.0)>,
 <gurobi.Var Transfer1[2,0] (value 0.0)>,
 <gurobi.Var Transfer1[2,1] (value 0.0)>,
 <gurobi.Var Transfer1[2,2] (value 0.0)>,
 <gurobi.Var Transfer1[3,0] (value 0.0)>,
 <gurobi.Var Transfer1[3,1] (value 0.0)>,
 <gurobi.Var Transfer1[3,2] (value 0.0)>,
 <gurobi.Var Transfer1[4,0] (value 0.0)>,
 <gurobi.Var Transfer1[4,1] (value 0.0)>,
 <gurobi.Var Transfer1[4,2] (value 0.0)>,
 <gurobi.Var Transfer2[0,0] (value 0.0)>,
 <gurobi.Var Transfer2[0,1] (value 200.0)>,
 <gurobi.Var Transfer2[1,0] (value 400.0)>,
 <gurobi.Var Transfer2[1,1] (value 0.0)>,
 <gurobi.Var Transfer2[2,0]

# Задание 4. Удельная стоимость хранения на складах

В целевую функцию нужно будет добавить массу, хранимую на складе, умноженную на удельную стоимость

In [149]:
C = np.array([2,1.5,3])

model = gp.Model('factoryProblem4')

x = model.addVars(5, ub=productivity, name='x')

Transfer1 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), 
                                        (1,0), (1,1), (1,2),
                                        (2,0), (2,1), (2,2),
                                        (3,0), (3,1), (3,2),
                                        (4,0), (4,1), (4,2)]), ub = D1, name = 'Transfer1')

Transfer2 = model.addVars(gp.tuplelist([(0,0), (0,1), 
                                        (1,0), (1,1), 
                                        (2,0), (2,1)]), name = 'Transfer2')

Transfer3 = model.addVars(gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                        (1,0), (1,1), (1,2), (1,3) ]), ub = D3, name = 'Transfer3')

factory_working = model.addVars(5, vtype=GRB.BINARY, name='factory-working')

for i in range(5):
    model.addConstr(x[i] <= 1000000*factory_working[i], name='factory-working')

In [150]:
model.addConstrs((x[i] == Transfer1.sum(i,'*') for i in range(5)), name = 'from factory to storage')

model.addConstrs((Transfer1.sum('*', i)
                 == Transfer2.sum(i, '*') for i in range(3)), name = 'from storage to sorting')

model.addConstrs((Transfer2.sum('*', i) 
                 == Transfer3.sum(i, '*') for i in range(2)), name = 'from sorting to buyers')

model.addConstrs((Transfer1.sum('*', i) <= storage_capacity[i] for i in range(3)), name = 'storage capacity')

model.addConstrs((Transfer3.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders full')

obj = gp.LinExpr()

obj += sum([x[i]*cost[i] for i in range(5)]) + sum([Transfer1[i, j]*C1[i, j] for i in range(5) for j in range(3)]) +sum([Transfer2[i, j]*C2[i, j] for i in range(3) for j in range(2)]) + sum([Transfer3[i, j]*C3[i, j] for i in range(2) for j in range(4)])

obj += sum([Transfer1.sum('*', i)*C[i] for i in range(3)])

model.setObjective(obj, GRB.MINIMIZE)

model.update()
model.display()

Minimize
   <gurobi.LinExpr: 1.3 x[0] + 1.5 x[1] + 1.2 x[2] + 1.7 x[3] + 1.1 x[4] + 3.0 Transfer1[0,0] + 3.5 Transfer1[0,1] + 3.0 Transfer1[0,2] + 4.0 Transfer1[1,0] + 1.5 Transfer1[1,1] + 4.0 Transfer1[1,2] + 5.0 Transfer1[2,0] + 4.5 Transfer1[2,1] + 3.0 Transfer1[2,2] + 2.0 Transfer1[3,0] + 4.5 Transfer1[3,1] + 5.0 Transfer1[3,2] + 4.0 Transfer1[4,0] + 1.5 Transfer1[4,1] + 6.0 Transfer1[4,2] + 2.0 Transfer2[0,0] + 3.0 Transfer2[0,1] + Transfer2[1,0] + 4.0 Transfer2[1,1] + 3.0 Transfer2[2,0] + Transfer2[2,1] + 2.0 Transfer3[0,0] + Transfer3[0,1] + 3.0 Transfer3[0,2] + 4.0 Transfer3[0,3] + 3.0 Transfer3[1,0] + 2.0 Transfer3[1,1] + 3.0 Transfer3[1,2] + Transfer3[1,3]>
Subject To
   factory-working : <gurobi.LinExpr: x[0] + -1000000.0 factory-working[0]> <= 0.0
   factory-working : <gurobi.LinExpr: x[1] + -1000000.0 factory-working[1]> <= 0.0
   factory-working : <gurobi.LinExpr: x[2] + -1000000.0 factory-working[2]> <= 0.0
   factory-working : <gurobi.LinExpr: x[3] + -1000000.0 factory-

In [151]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 39 columns and 88 nonzeros
Model fingerprint: 0xf9fea5db
Variable types: 34 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 6e+00]
  Bounds range     [1e+00, 7e+02]
  RHS range        [1e+02, 5e+02]
Presolve removed 14 rows and 23 columns
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 34 nonzeros
Variable types: 16 continuous, 0 integer (0 binary)

Root relaxation: objective 8.990000e+03, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8990.0000000 8990.00000  0.00%     -    0s

Explored 0 nodes (9 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution cou